<a href="https://colab.research.google.com/github/ja754969/OC_2021_project/blob/practice-and-read/NCCU_yenlung_Deep-Learning/05_2_%E4%BD%9C%E6%A5%AD%E8%AA%AA%E6%98%8E_%E8%BD%89%E7%A7%BB%E5%AD%B8%E7%BF%92%E7%9A%84%E7%B7%B4%E7%BF%92_(%E4%BB%A5_CNN_%E6%89%8B%E5%AF%AB%E8%BE%A8%E8%AD%98%E7%82%BA%E4%BE%8B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 主題 05-2. 轉移學習的練習

【註】因 TensorFlow 2 已做了一些改變, 例如完全整合了 Keras。到 2021 年的今天, 有一些細節也做了調整。因此我們依新的規範修改了程式。最大的不同是, 以後大家直接安裝 tensorflow 即可, 不用再另外裝 keras。

讓我們回顧一下生命中第一個做出來的 CNN 圖形辨識模型...

## 1. 初始準備

In [12]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# tf.Keras functions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD

# tf.Keras dataset
from tensorflow.keras.datasets import mnist

# Keras utilis function
from tensorflow.keras.utils import to_categorical

## 2. 讀入 MNIST 數據庫

MNIST 是有一堆 0-9 的手寫數字圖庫。有 6 萬筆訓練資料, 1 萬筆測試資料。它是 "Modified" 版的 NIST 數據庫, 原來的版本有更多資料。這個 Modified 的版本是由 LeCun, Cortes, 及 Burges 等人做的。可以參考這個數據庫的[原始網頁](http://yann.lecun.com/exdb/mnist/)。

MNIST 可以說是 Deep Learning 最有名的範例, 它被 Deep Learning 大師 Hinton 稱為「機器學習的果蠅」。

### 2.1 由 tf.Keras 讀入 MNIST
tf.Keras 很貼心的幫我們準備好 MNIST 數據庫, 我們可以這樣讀進來 (第一周課程中已經讀過)。

In [14]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

我們可以看看資料的長相

In [15]:
print("There are %d training data with size %d x %d" %x_train.shape)
print("There are %d testing  data with size %d x %d" %x_test.shape)

There are 60000 training data with size 28 x 28
There are 10000 testing  data with size 28 x 28


### 2.3 輸入格式整理

我們現在要用 CNN 學手寫辨識。因為 CNN 模型的資料需要多一個 channel (通道數)，因此我們要用 `reshape` 調校一下。

In [16]:
x_train = x_train.reshape(60000, 28, 28, 1)/255
x_test = x_test.reshape(10000, 28, 28, 1)/255

為了後面需要，我們先將數字 0 和 1 的資料分別抓出來

In [17]:
x_train_01 = x_train[y_train <= 1]
x_test_01 = x_test[y_test <= 1]

並將 label 轉換成 one-hot encoding 的形式

In [18]:
y_train_10 = to_categorical(y_train, 10)
y_test_10 = to_categorical(y_test, 10)

y_train_01 = y_train[y_train <= 1]
y_train_01 = to_categorical(y_train_01, 2)

y_test_01 = y_test[y_test <= 1]
y_test_01 = to_categorical(y_test_01, 2)

養成良好的習慣，適時的確認資料的大小以確保資料的一致性

In [19]:
x_train_01.shape, x_test_01.shape

((12665, 28, 28, 1), (2115, 28, 28, 1))

In [20]:
y_train_01.shape, y_test_01.shape

((12665, 2), (2115, 2))

# 3. 回顧 CNN 圖形辨識模型

經典的 CNN 圖形辨識模型 LeNet-5 是一個由兩層卷積層加三層全連接層所建立的神經網路，而在第二單元時，我們建立的 CNN 模型設定如下：

* 起始為 <span style="color:red;">3</span> 個 convolutional block
 + 每個 convolutional block 為 <span style="color:red;">1</span> 個 2D Convolution + ReLU + <span style="color:red;">1</span> 個 2D MaxPooling
 + 2D Convolution 的數量為 32, 64, 128
 + 每個 2D Convolution 的 `kernal_size` 為 3 或 (3, 3)，`padding` 使用 `same`
 + 每個 2D MaxPooling 的 `pool_size` 為 2 或 (2, 2)，`padding` 使用 `same`

* 將輸出結果 `Flatten` 後，接著兩層全連接層，神經元個數分別為 200 和 10 (<span style="color:red;">數字的類別總數)</span>

我們當時建立的，是一個具有三層卷積層加兩層全連接的神經網路，其實可以看成是 LeNet-5 的一種變形。

根據本單元的內容，我們可以使用下列方式使用 Sequential 重新建構第二單元的 CNN 模型。

In [21]:
# We put 3 conv. blocks together, called conv_layer.
conv_layer = [Conv2D(32, (3, 3), padding='same', input_shape=(28,28,1)),
              Activation('relu'),
              MaxPooling2D(pool_size=(2, 2)),
              
              Conv2D(64, (3, 3), padding='same'),
              Activation('relu'),
              MaxPooling2D(pool_size=(2, 2)),
              
              Conv2D(128, (3, 3), padding='same'),
              Activation('relu'),
              MaxPooling2D(pool_size=(2, 2))]

# We put Flatten, and 2 fully-connectd layers together, called fc_layer.
fc_layer = [Flatten(),
            Dense(200),
            Activation('relu'),
            Dense(10),
            Activation('softmax')]

model = Sequential(conv_layer + fc_layer)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_5 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_6 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 128)        

In [22]:
model.load_weights('handwriting_weights_cnn.h5')

# 4. 保留前三層 convolutional layer 並進行轉移學習

在此，我們一樣將 MNIST 資料集將僅有 0, 1的部分取出來，我們希望透過轉移學習建立一個類似 LeNet-5 的 0, 1 圖形辨識模型。

請將下列三個 **None** 的部分進行修改，以透過轉移學習建立新的模型。

In [24]:
new_fc_layer = [Flatten(), 
                ### Design your own fully connected structures ###
                Dense(200),
                Activation('sigmoid'),
                Dense(2),       ## Hint: how many classes in new dataset?
                ### Remember put correct number of unit for output ###
                Activation('softmax')]

model_0_to_1 = Sequential(conv_layer + new_fc_layer)
model_0_to_1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_5 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_6 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 128)        

請將下列的 **None** 進行修改，以將借過來的神經網路 **冷凍** 起來：

In [25]:
for layer in conv_layer:
    layer.trainable = False

**冷凍**後的神經網路的 summary 會有些變化，你有發現嗎? ：)

In [26]:
model_0_to_1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_5 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_6 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 128)        

接著，我們來訓練這個(有一部分架構及權重跟別人借用的) 0, 1 手寫辨識模型吧！

In [27]:
model_0_to_1.compile(loss='mse', optimizer=SGD(learning_rate=0.1), metrics=['accuracy'])

## 5. 訓練

In [28]:
model_0_to_1.fit(x_train_01, y_train_01, batch_size=100, epochs=5)

Epoch 1/5
127/127 [==============================] - 7s 47ms/step - loss: 0.0298 - accuracy: 0.9619
Epoch 2/5
127/127 [==============================] - 6s 45ms/step - loss: 0.0047 - accuracy: 0.9956
Epoch 3/5
127/127 [==============================] - 6s 46ms/step - loss: 0.0036 - accuracy: 0.9961
Epoch 4/5
127/127 [==============================] - 6s 46ms/step - loss: 0.0030 - accuracy: 0.9972
Epoch 5/5
127/127 [==============================] - 6s 45ms/step - loss: 0.0026 - accuracy: 0.9974


In [29]:
score = model_0_to_1.evaluate(x_test_01, y_test_01)

67/67 [==============================] - 1s 16ms/step - loss: 0.0016 - accuracy: 0.9981


In [30]:
print('測試資料的 loss:', score[0])
print('測試資料正確率:', score[1])

測試資料的 loss: 0.0015654639573767781
測試資料正確率: 0.9981087446212769


## 6. 恭喜你完成了第二個透過轉移學習得到的神經網路模型！

不難發現，如果模型大部分的權重已經訓練好並冷凍起來，則轉移學習可以大幅減少訓練時間且訓練會更快收斂，那麼，是否還有其他重要的模型建構技巧呢？

這個問題我們留待下個單元解答囉~ : )